In [2]:
import pandas as pd
import numpy as np
import torch
import os
import shutil
import sys
from PIL import Image
import utility_functions as uf

# testing area

In [ ]:
#check if all the slides are processed to patches, and find the number of patches is less than 10000
directory_path = '/home/exon_storage1/aml_slide/patches/'
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        number_of_patches = len(os.listdir(patches_path))
        if number_of_patches <= 10000:
            print( file+" : "+str(len(os.listdir(patches_path))))
    else:
        print(file+" is not processed yet")

In [8]:
#this is a script to remove the slides with the suffix of x or Rx, and rename the slides with the suffix of Lx
path = "/home/exon_storage1/aml_slide/renameByUPN/"
for file in os.listdir(path):
    detector = file.split("_")[-1]
    detector = detector.split(".")[0]
    if detector =="Lx":
        oldname = path+file
        newname = oldname.replace("_Lx","")
        print(oldname)
        os.rename(oldname, newname)
    elif detector == "x" or detector == "Rx":
        print(path+file)
        os.remove(path+file)
        # shutil.rmtree(path+file)

/home/exon_storage1/aml_slide/renameByUPN/A877_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A909_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A152_Lx.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A503_Rx.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A483_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1052_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A910_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A823_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A529_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A927_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A481_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A201_Lx.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1163_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1050_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A458_Rx.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A470_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1152_x.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1020_Rx.ndpi
/home/exon_storage

In [ ]:
#this script is used to normalize the patches and save them in a new folder, with a blurry filter
sys.path.insert(1, '/home/weber50432/AML_image_processing/HEnorm_python/')
from normalizeStaining import normalizeStaining , is_blurry
directory_path = '/home/weber50432/AML_image_processing/PyHIST/output/'
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        if os.path.exists(directory_path+file+"/rightside_patch_norm"):
            print(file + " rightside_patch_norm is already created !")
            continue
        else:
            os.mkdir(directory_path+file+"/rightside_patch_norm/")
        if os.path.exists(directory_path+file+"/rightside_patch_blurry"):
            print(file + " rightside_patch_blurry is already created !")
        else:
            os.mkdir(directory_path+file+"/rightside_patch_blurry/")
        patches_path_blurry = directory_path+file+"/rightside_patch_blurry/"
        patches_path_norm = directory_path+file+"/rightside_patch_norm/"
        for img in os.listdir(patches_path):
            if img.endswith(".png"):
                img_num = str(img)
                img_path = patches_path+img
                img = np.array(Image.open(img_path))
                Inorm, H, E = normalizeStaining(img = img,
                      saveFile = None,
                      Io = 240,
                      alpha = 1,
                      beta = 0.15)
                if is_blurry(Inorm):
                    Image.fromarray(Inorm).save(patches_path_blurry+img_num)
                    # os.remove(img_path)
                    print(img_num+" is blurry !")
                else:
                    Image.fromarray(Inorm).save(patches_path_norm+img_num)
                    print(img_num+" is saved !")

In [ ]:
patches_size = 256
slide =13
path1 = '/home/exon_storage1/aml_slide/otsu_0.9/A{}/tile_selection.tsv'.format(slide)
path2 = '/home/exon_storage1/aml_slide/otsu_1.0/A{}/tile_selection.tsv'.format(slide)
list1, num1 = uf.get_coordinates(path1,patches_size,0.6,0.2)
list2, num2 = uf.get_coordinates(path2,patches_size,0.6,0.2)
print(list1[:5])
print(list2[:5])
print(num1)
print(num2)
set1 = set(list1)
set2 = set(list2)
difference = set1.difference(set2)
result = list(difference)
print(result)
print(len(result),type(result[0]))
grid = []
grid.append([(1,2),(2,3)])
grid.append([(2,2),(2,3)])
print(grid[:2][:5])
output = {
    "slides": ["my/full/path/slide01.svs", "my/full/path/slide02.svs"],
    "grid": grid
    ,
    "targets": [0, 1],
    "mult": patches_size/224,
    "level": 0,
}

# torch.save(output, "./data.pt")


In [ ]:
lib = torch.load("./data.pt")
grid = []
slideIDX = []
for i,g in enumerate(lib['grid']):
  grid.extend(g)
  slideIDX.extend([i]*len(g))

print('Number of tiles: {}'.format(len(grid)))

In [ ]:
list1 = [(40448, 240640), (40448, 241152), (40448, 241664), (40448, 242176), (40448, 242688)]
list2 = [(40448, 240640), (40448, 241152), (40448, 241664), (404481, 242176), (40448, 242688)]

set1 = set(list1)
set2 = set(list2)

difference = set2.difference(set1)

result = list(difference)

print(result)